# 说明
用于实现AAnet的训练和测试,使用lightning框架

## 加载数据集并划分训练集和验证集
需要修改的地方：
- file_dir: 数据集的路径
- gesture_classes: 手势的类别数
- sample_num: 每个类别的样本数


In [1]:
#tensorboard
import tensorboard

In [1]:
import scipy.io as sio
import torch
from sklearn.preprocessing import OneHotEncoder
from torchsummary import summary
import torch.utils.data as data
import lightning as L
from lightning.pytorch import seed_everything

seed_everything(42, workers=True)   # 固定随机种子

file_dir = 'out\\pyfeature\\'
file_dir = 'out\\test\\pyfeature\\'
file_dir = 'out\\1000\\'
gesture_classes = 4
sample_num = 3000

def load_data(file_path, gesture_index):
    """加载数据"""
    filename = file_path + str(gesture_index) + '.mat'
    print(filename)
    data = sio.loadmat(filename)
    range_profile = torch.tensor(data['range_profile'], dtype=torch.float32)
    speed_profile = torch.tensor(data['speed_profile'], dtype=torch.float32)
    angle_profile = torch.tensor(data['angle_profile'], dtype=torch.float32)
    return range_profile, speed_profile, angle_profile

def generate_labels(gesture_class, sample_num):
    """生成标签"""
    labels = torch.zeros((gesture_class * sample_num, 1))
    for i in range(gesture_class):
        labels[i * sample_num:(i + 1) * sample_num] = i
    enc = OneHotEncoder()
    labels = enc.fit_transform(labels).toarray()
    # 加两列0，使得标签的维度和输出的维度一致
    # labels = torch.cat((torch.tensor(labels, dtype=torch.float32), torch.zeros((gesture_class * sample_num, 2), dtype=torch.float32)), 1)
    # 加6列0，使得标签的维度和输出的维度一致
    labels = torch.cat((torch.tensor(labels, dtype=torch.float32), torch.zeros((gesture_class * sample_num, 6), dtype=torch.float32)), 1)
    
    return torch.tensor(labels, dtype=torch.float32)

# 初始化数据集
range_profile = torch.zeros((gesture_classes * sample_num, 30, 64), dtype=torch.float32)
speed_profile = torch.zeros((gesture_classes * sample_num, 30, 64), dtype=torch.float32)
angle_profile = torch.zeros((gesture_classes * sample_num, 30, 64), dtype=torch.float32)

labels = generate_labels(gesture_classes, sample_num)   # 生成标签
print(labels.shape)
# 加载数据
for i in range(gesture_classes):
    range_profile[i * sample_num:(i + 1) * sample_num, :, :], \
    speed_profile[i * sample_num:(i + 1) * sample_num, :, :], \
    angle_profile[i * sample_num:(i + 1) * sample_num, :, :] = load_data(file_dir, i + 1)

print(range_profile.shape)

range_profile, speed_profile, angle_profile, labels = load_data(file_dir, ['avg', 'mti', 'None'])

dataset_loader = data.TensorDataset(range_profile, speed_profile, angle_profile, labels)

# 将数据划分为训练集和验证集
# train_loader, val_loader= data.random_split(dataset_loader, [0.8, 0.2])
# 将数据划分为训练集,验证集和测试集
train_loader, val_loader, test_loader = data.random_split(dataset_loader, [0.7, 0.2, 0.1])

Seed set to 42
C:\Users\Du\AppData\Local\Temp\ipykernel_11788\74597640.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(labels, dtype=torch.float32)


torch.Size([12000, 10])
out\1000\1.mat


FileNotFoundError: [Errno 2] No such file or directory: 'out\\1000\\1.mat'

In [110]:
import scipy.io as sio
import torch
from sklearn.preprocessing import OneHotEncoder
from torchsummary import summary
import torch.utils.data as data
import lightning as L
from lightning.pytorch import seed_everything

seed_everything(42, workers=True)   # 固定随机种子

# file_dir = 'out\\pyfeature\\'
# file_dir = 'out\\test\\pyfeature\\'
# file_dir = 'out\\1000\\'
# gesture_classes = 4
# # 目标类别数
# target_classes = 10
# # 子文件夹
# sub_dir_list = ['avg', 'mti', 'None']
# # 子文件夹长度
# sub_dir_len = len(sub_dir_list)
# # 样本数
# sample_num = 1000

# 设置上面5个参数的函数
def set_params(file_dir, gesture_classes, target_classes, sub_dir_list, sample_num):
    return file_dir, gesture_classes, target_classes, sub_dir_list, sample_num

def load_data(file_name):
    """加载数据"""
    data = sio.loadmat(file_name)
    # range_profile = torch.tensor(data['range_profile'], dtype=torch.float32)
    # speed_profile = torch.tensor(data['speed_profile'], dtype=torch.float32)
    # angle_profile = torch.tensor(data['angle_profile'], dtype=torch.float32)
    
    # 因为数据原本是complex类型，所以要先取出复数数据,再转化为float32
    # range_profile = torch.tensor(data['range_profile'], dtype=torch.complex128)
    range_profile = torch.tensor(data['range_profile'], dtype=torch.complex128).abs().float()
    speed_profile = torch.tensor(data['speed_profile'], dtype=torch.complex128).abs().float()
    angle_profile = torch.tensor(data['angle_profile'], dtype=torch.complex128).abs().float()
    
    return range_profile, speed_profile, angle_profile

def generate_labels(gesture_class, class_sample_num):
    """生成标签"""
    labels = torch.zeros((gesture_class * class_sample_num, 1))
    for i in range(gesture_class):
        labels[i * class_sample_num:(i + 1) * class_sample_num] = i
    enc = OneHotEncoder()
    labels = enc.fit_transform(labels).toarray()
    # 加6列0，使得标签的维度和输出的维度一致
    labels = torch.cat((torch.tensor(labels, dtype=torch.float32), torch.zeros((gesture_class * class_sample_num, target_classes-gesture_class), dtype=torch.float32)), 1)
    return torch.tensor(labels, dtype=torch.float32)


# 写data_loader函数，传入文件夹路径，和子文件夹名称列表如(avg, mti, None),返回range_profile, speed_profile, angle_profile, labels
def data_loader(file_dir, sub_dir_list):
    sub_dir_len = len(sub_dir_list)
    
    range_profile = torch.zeros((gesture_classes * sample_num*sub_dir_len, 30, 64), dtype=torch.float32)
    speed_profile = torch.zeros((gesture_classes * sample_num*sub_dir_len, 30, 64), dtype=torch.float32)
    angle_profile = torch.zeros((gesture_classes * sample_num*sub_dir_len, 30, 64), dtype=torch.float32)
    labels = generate_labels(gesture_classes, sample_num*sub_dir_len)   # 生成标签
    for i in range(gesture_classes):
        for j in range(sub_dir_len):
            filename = file_dir + sub_dir_list[j] + '/' + str(i + 1) + '.mat'
            print("当前i:",i,"当前j:",j,"读取文件：",filename,"将存放在：",i*sample_num*sub_dir_len+j*sample_num,"到",i*sample_num*sub_dir_len+(j + 1)*sample_num,"之间")
            range_profile[i*sample_num*sub_dir_len+j*sample_num:i*sample_num*sub_dir_len+(j + 1)*sample_num, :, :], \
            speed_profile[i*sample_num*sub_dir_len+j*sample_num:i*sample_num*sub_dir_len+(j + 1)*sample_num, :, :], \
            angle_profile[i*sample_num*sub_dir_len+j*sample_num:i*sample_num*sub_dir_len+(j + 1)*sample_num, :, :] = load_data(filename)
            
    
    return range_profile, speed_profile, angle_profile, labels


file_dir, gesture_classes, target_classes, sub_dir_list, sample_num = set_params('out\\1000\\', 5, 5, ['avg','mti','None'], 1000)
range_profile, speed_profile, angle_profile, labels = data_loader(file_dir, sub_dir_list)

print(range_profile.shape)
print(labels.shape)

dataset_loader = data.TensorDataset(range_profile, speed_profile, angle_profile, labels)

# # 将数据划分为训练集和验证集
# # train_loader, val_loader= data.random_split(dataset_loader, [0.8, 0.2])
# 将数据划分为训练集,验证集和测试集
train_loader, val_loader, test_loader = data.random_split(dataset_loader, [0.7, 0.2, 0.1])

# file_dir, gesture_classes, target_classes, sub_dir_list, sample_num = set_params('out\\1000\\', 4, 10, ['avg_remove'], 2000)
# train_range_profile, train_speed_profile, train_angle_profile, train_labels = data_loader(r"K:/aio_radar/out/1000/", ['avg_remove'])
# dataset_loader = data.TensorDataset(train_range_profile, train_speed_profile, train_angle_profile, train_labels)
# train_loader, val_loader = data.random_split(dataset_loader, [0.8, 0.2])

# file_dir, gesture_classes, target_classes, sub_dir_list, sample_num = set_params('out\\1000\\', 4, 10, ['avg'], 1000)
# test_range_profile, test_speed_profile, test_angle_profile, test_labels = data_loader(file_dir, sub_dir_list)
# test_loader = data.TensorDataset(test_range_profile, test_speed_profile, test_angle_profile, test_labels)

Seed set to 42
C:\Users\Du\AppData\Local\Temp\ipykernel_11788\1566842790.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(labels, dtype=torch.float32)


当前i: 0 当前j: 0 读取文件： out\1000\avg/1.mat 将存放在： 0 到 1000 之间
当前i: 0 当前j: 1 读取文件： out\1000\mti/1.mat 将存放在： 1000 到 2000 之间
当前i: 0 当前j: 2 读取文件： out\1000\None/1.mat 将存放在： 2000 到 3000 之间
当前i: 1 当前j: 0 读取文件： out\1000\avg/2.mat 将存放在： 3000 到 4000 之间
当前i: 1 当前j: 1 读取文件： out\1000\mti/2.mat 将存放在： 4000 到 5000 之间
当前i: 1 当前j: 2 读取文件： out\1000\None/2.mat 将存放在： 5000 到 6000 之间
当前i: 2 当前j: 0 读取文件： out\1000\avg/3.mat 将存放在： 6000 到 7000 之间
当前i: 2 当前j: 1 读取文件： out\1000\mti/3.mat 将存放在： 7000 到 8000 之间
当前i: 2 当前j: 2 读取文件： out\1000\None/3.mat 将存放在： 8000 到 9000 之间
当前i: 3 当前j: 0 读取文件： out\1000\avg/4.mat 将存放在： 9000 到 10000 之间
当前i: 3 当前j: 1 读取文件： out\1000\mti/4.mat 将存放在： 10000 到 11000 之间
当前i: 3 当前j: 2 读取文件： out\1000\None/4.mat 将存放在： 11000 到 12000 之间
当前i: 4 当前j: 0 读取文件： out\1000\avg/5.mat 将存放在： 12000 到 13000 之间
当前i: 4 当前j: 1 读取文件： out\1000\mti/5.mat 将存放在： 13000 到 14000 之间
当前i: 4 当前j: 2 读取文件： out\1000\None/5.mat 将存放在： 14000 到 15000 之间
torch.Size([15000, 30, 64])
torch.Size([15000, 5])


In [94]:
# 定义模型
# --------------------------------
# 步骤 2: 定义 RadarGestureNet
# --------------------------------
from sklearn.preprocessing import OneHotEncoder
import torch.nn as nn
def one_hot_labels(caategorical_labels):
    enc = OneHotEncoder(handle_unknown='ignore')
    on_hot_labels = enc.fit_transform(
        caategorical_labels.reshape(-1, 1)).toarray()
    return on_hot_labels
def one_hot_to_label(one_hot):
    return torch.argmax(one_hot, dim=1)

encoder = nn.Sequential(
            nn.LayerNorm([30, 64]),
            nn.Conv1d(30, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(16, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(8, 4, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, gesture_classes),
        )


class RadarGestureNet(L.LightningModule):
    def __init__(self, encoder, gesture_class):
        super().__init__()
        self.gesture_class = gesture_class
        self.save_hyperparameters()
        self.encoder = encoder
    def forward(self, x):
        embedding = self.encoder(x)
        return embedding
    def training_step(self, batch, batch_idx):
        x1,x2,x3, y = batch
        z = self.encoder(x2)+self.encoder(x3)
        criterion = nn.MSELoss()
        loss = criterion(z, y)
        self.log("train_loss", loss)
        train_accuracy = torch.sum(one_hot_to_label(z) == one_hot_to_label(y)).item() / len(y)
        self.log("train_accuracy", train_accuracy)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x1,x2,x3, y = batch
        z = self.encoder(x2)+self.encoder(x3)
        criterion = nn.MSELoss()
        val_loss = criterion(z, y)
        self.log("val_loss", val_loss)
    
    def test_step(self, batch, batch_idx):
        x1,x2,x3, y = batch
        z = self.encoder(x2)+self.encoder(x3)
        criterion = nn.MSELoss()
        
        test_loss = criterion(z, y)
        self.log("test_loss", test_loss)
        
        accuracy = torch.sum(one_hot_to_label(z) == one_hot_to_label(y)).item() / len(y)
        self.log("accuracy", accuracy)
        
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        return self(batch)
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=3e-3)
        return optimizer


In [24]:
# 定义模型(临时)
# --------------------------------
# 步骤 2: 定义 RadarGestureNet
# --------------------------------
from sklearn.preprocessing import OneHotEncoder
import torch.nn as nn
def one_hot_labels(caategorical_labels):
    enc = OneHotEncoder(handle_unknown='ignore')
    on_hot_labels = enc.fit_transform(
        caategorical_labels.reshape(-1, 1)).toarray()
    return on_hot_labels
def one_hot_to_label(one_hot):
    return torch.argmax(one_hot, dim=1)

class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionModule, self).__init__()
        self.branch1x1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.branch3x3 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.branch5x5 = nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=2)
        self.branch_pool = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        branch3x3 = self.branch3x3(x)
        branch5x5 = self.branch5x5(x)
        branch_pool = self.branch_pool(nn.functional.max_pool1d(x, kernel_size=3, stride=1, padding=1))
        
        outputs = [branch1x1, branch3x3, branch5x5, branch_pool]
        
        outputs = torch.cat(outputs, 1)  # Concatenate along the channel dimension
        return outputs

class RadarGestureNet(L.LightningModule):
    def __init__(self, gesture_class):
        super().__init__()
        self.gesture_class = gesture_class
        self.save_hyperparameters('gesture_class')
        
        self.Icp1 = nn.Sequential(
            nn.LayerNorm([30, 64]),
            nn.Dropout(0.6),
            InceptionModule(30, 64)
        )
        
        self.Icp2 = nn.Sequential(
            nn.LayerNorm([30, 64]),
            nn.Dropout(0.6),
            InceptionModule(30, 64)
        )
        self.Icp3 = nn.Sequential(
            nn.LayerNorm([30, 64]),
            nn.Dropout(0.6),
            InceptionModule(30, 64)
        )
        
        self.decoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16384, 512),
            nn.Dropout(0.75),
            nn.ReLU(),
            nn.Linear(512, 64),
            nn.Dropout(0.75),
            nn.ReLU(),
            nn.Linear(64, 10),
            nn.Softmax(dim=1)
        )
    def forward(self, x1, x2, x3):
        embedding = self.Icp1(x1)+self.Icp2(x2)+self.Icp3(x3)
        
        embedding = self.decoder(embedding)
        return embedding
    def training_step(self, batch, batch_idx):
        x1,x2,x3, y = batch
        z = self.forward(x1, x2, x3)
        criterion = nn.MSELoss()
        loss = criterion(z, y)
        self.log("train_loss", loss)
        train_accuracy = torch.sum(one_hot_to_label(z) == one_hot_to_label(y)).item() / len(y)
        self.log("train_accuracy", train_accuracy)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x1,x2,x3, y = batch
        z = self.forward(x1, x2, x3)
        criterion = nn.MSELoss()
        val_loss = criterion(z, y)
        self.log("val_loss", val_loss)
    
    def test_step(self, batch, batch_idx):
        x1,x2,x3, y = batch
        z = self.forward(x1, x2, x3)
        criterion = nn.MSELoss()
        
        test_loss = criterion(z, y)
        self.log("test_loss", test_loss)
        
        accuracy = torch.sum(one_hot_to_label(z) == one_hot_to_label(y)).item() / len(y)
        self.log("accuracy", accuracy)
        
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        return self(batch)
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer

model = RadarGestureNet(gesture_class=gesture_classes)
summary(model, input_size=[(30, 64),(30, 64),(30, 64)],device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         LayerNorm-1               [-1, 30, 64]           3,840
           Dropout-2               [-1, 30, 64]               0
            Conv1d-3               [-1, 64, 64]           1,984
            Conv1d-4               [-1, 64, 64]           5,824
            Conv1d-5               [-1, 64, 64]           9,664
            Conv1d-6               [-1, 64, 64]           1,984
   InceptionModule-7              [-1, 256, 64]               0
         LayerNorm-8               [-1, 30, 64]           3,840
           Dropout-9               [-1, 30, 64]               0
           Conv1d-10               [-1, 64, 64]           1,984
           Conv1d-11               [-1, 64, 64]           5,824
           Conv1d-12               [-1, 64, 64]           9,664
           Conv1d-13               [-1, 64, 64]           1,984
  InceptionModule-14              [-1, 

In [16]:
# 定义模型(深度可分离)
# --------------------------------
# 步骤 2: 定义 RadarGestureNet
# --------------------------------
from sklearn.preprocessing import OneHotEncoder
import torch.nn as nn
def one_hot_labels(caategorical_labels):
    enc = OneHotEncoder(handle_unknown='ignore')
    on_hot_labels = enc.fit_transform(
        caategorical_labels.reshape(-1, 1)).toarray()
    return on_hot_labels
def one_hot_to_label(one_hot):
    return torch.argmax(one_hot, dim=1)

encoder = 0

#
class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionModule, self).__init__()
        self.branch1x1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.branch3x3 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.branch5x5 = nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=2)
        self.branch_pool = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        branch3x3 = self.branch3x3(x)
        branch5x5 = self.branch5x5(x)
        branch_pool = self.branch_pool(nn.functional.max_pool1d(x, kernel_size=3, stride=1, padding=1))
        
        outputs = [branch1x1, branch3x3, branch5x5, branch_pool]
        
        outputs = torch.cat(outputs, 1)
        return outputs

# 深度可分离卷积
class DepthwiseSeparableConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super(DepthwiseSeparableConv1d, self).__init__()
        self.depthwise = nn.Conv1d(in_channels, in_channels, kernel_size, groups=in_channels, padding=padding)
        self.pointwise = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x




class ABCModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ABCModule, self).__init__()
        self.branch3x3 = nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=1)

    def forward(self, x):
        branch3x3 = self.branch3x3(x)
        
        outputs = [branch3x3]
        
        outputs = torch.cat(outputs, 1)  # Concatenate along the channel dimension
        return outputs

class RadarGestureNet(L.LightningModule):
    def __init__(self, gesture_class):
        super().__init__()
        self.gesture_class = gesture_class
        self.save_hyperparameters()
        
        self.ABC1 = nn.Sequential(
            nn.LayerNorm([30, 64]),
            DepthwiseSeparableConv1d(30, 15, 3, padding=1),
            InceptionModule(15, 5),
        )
        
        self.ABC2 = nn.Sequential(
            nn.LayerNorm([30, 64]),
            DepthwiseSeparableConv1d(30, 15, 3, padding=1),
            InceptionModule(15, 5),
        )
        
        self.ABC3 = nn.Sequential(
            nn.LayerNorm([30, 64]),
            DepthwiseSeparableConv1d(30, 15, 3, padding=1),
            InceptionModule(15, 5),
        )
        
        
        self.decoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1280, 640),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(640, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
            nn.Softmax(dim=1)
        )
    def forward(self, x1, x2, x3):
        embedding = (self.ABC1(x1)+self.ABC2(x3))*self.ABC3(x2)
        
        embedding = self.decoder(embedding)
        return embedding
    def training_step(self, batch, batch_idx):
        x1,x2,x3, y = batch
        z = self.forward(x1, x2, x3)
        criterion = nn.MSELoss()
        
        loss = criterion(z, y)
        self.log("train_loss", loss)
        train_accuracy = torch.sum(one_hot_to_label(z) == one_hot_to_label(y)).item() / len(y)
        self.log("train_accuracy", train_accuracy)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x1,x2,x3, y = batch
        z = self.forward(x1, x2, x3)
        criterion = nn.MSELoss()
        val_loss = criterion(z, y)
        self.log("val_loss", val_loss)
    
    def test_step(self, batch, batch_idx):
        x1,x2,x3, y = batch
        z = self.forward(x1, x2, x3)
        criterion = nn.MSELoss()
        test_loss = criterion(z, y)
        self.log("test_loss", test_loss)
        
        accuracy = torch.sum(one_hot_to_label(z) == one_hot_to_label(y)).item() / len(y)
        self.log("accuracy", accuracy)
        
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        return self(batch)
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer

model = RadarGestureNet( gesture_class=gesture_classes)
summary(model, input_size=[(30, 64),(30, 64),(30, 64)],device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         LayerNorm-1               [-1, 30, 64]           3,840
            Conv1d-2               [-1, 30, 64]             120
            Conv1d-3               [-1, 15, 64]             465
DepthwiseSeparableConv1d-4               [-1, 15, 64]               0
            Conv1d-5                [-1, 5, 64]              80
            Conv1d-6                [-1, 5, 64]             230
            Conv1d-7                [-1, 5, 64]             380
            Conv1d-8                [-1, 5, 64]              80
   InceptionModule-9               [-1, 20, 64]               0
        LayerNorm-10               [-1, 30, 64]           3,840
           Conv1d-11               [-1, 30, 64]             120
           Conv1d-12               [-1, 15, 64]             465
DepthwiseSeparableConv1d-13               [-1, 15, 64]               0
           Conv1d-14      

In [146]:
# 定义模型(深度可分离)2024_4_3
# --------------------------------
# 步骤 2: 定义 RadarGestureNet
# --------------------------------
from sklearn.preprocessing import OneHotEncoder
import torch.nn as nn
def one_hot_labels(caategorical_labels):
    enc = OneHotEncoder(handle_unknown='ignore')
    on_hot_labels = enc.fit_transform(
        caategorical_labels.reshape(-1, 1)).toarray()
    return on_hot_labels
def one_hot_to_label(one_hot):
    return torch.argmax(one_hot, dim=1)

encoder = 0

class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionModule, self).__init__()
        self.branch1x1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.branch3x3 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.branch5x5 = nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=2)
        self.branch_pool = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        branch3x3 = self.branch3x3(x)
        branch5x5 = self.branch5x5(x)
        branch_pool = self.branch_pool(nn.functional.max_pool1d(x, kernel_size=3, stride=1, padding=1))
        
        outputs = [branch1x1, branch3x3, branch5x5, branch_pool]
        
        outputs = torch.cat(outputs, 1)
        return outputs

# 深度可分离卷积
class DepthwiseSeparableConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super(DepthwiseSeparableConv1d, self).__init__()
        self.depthwise = nn.Conv1d(in_channels, in_channels, kernel_size, groups=in_channels, padding=padding)
        self.pointwise = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class ABCModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ABCModule, self).__init__()
        self.branch3x3 = nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=1)

    def forward(self, x):
        branch3x3 = self.branch3x3(x)
        
        outputs = [branch3x3]
        
        outputs = torch.cat(outputs, 1)  # Concatenate along the channel dimension
        return outputs

class RadarGestureNet(L.LightningModule):
    def __init__(self, gesture_class):
        super().__init__()
        self.gesture_class = gesture_class
        self.save_hyperparameters()
        
        self.ABC1 = nn.Sequential(
            nn.LayerNorm([30, 64]),
            DepthwiseSeparableConv1d(30, 15, 3, padding=1),
            nn.ReLU(),
            InceptionModule(15, 10),
            nn.ReLU(),
        )
        
        self.ABC2 = nn.Sequential(
            nn.LayerNorm([30, 64]),
            DepthwiseSeparableConv1d(30, 15, 3, padding=1),
            nn.ReLU(),
            InceptionModule(15, 10),
            nn.ReLU(),
        )
        
        self.ABC3 = nn.Sequential(
            nn.LayerNorm([30, 64]),
            DepthwiseSeparableConv1d(30, 15, 3, padding=1),
            nn.ReLU(),
            InceptionModule(15, 10),
            nn.ReLU(),
        )
        
        
        self.decoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2560, 640),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(640, 128),
            nn.ReLU(),
            nn.Linear(128, 5),
            nn.Softmax(dim=1)
        )
    def forward(self, x1, x2, x3):
        embedding = self.ABC1(x2)*(self.ABC2(x1)+self.ABC3(x3))
        
        embedding = self.decoder(embedding)
        return embedding
    def training_step(self, batch, batch_idx):
        x1,x2,x3, y = batch
        z = self.forward(x1, x2, x3)
        criterion = nn.MSELoss()
        
        loss = criterion(z, y)
        self.log("train_loss", loss)
        train_accuracy = torch.sum(one_hot_to_label(z) == one_hot_to_label(y)).item() / len(y)
        self.log("train_accuracy", train_accuracy)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x1,x2,x3, y = batch
        z = self.forward(x1, x2, x3)
        criterion = nn.MSELoss()
        val_loss = criterion(z, y)
        self.log("val_loss", val_loss)
    
    def test_step(self, batch, batch_idx):
        x1,x2,x3, y = batch
        z = self.forward(x1, x2, x3)
        criterion = nn.MSELoss()
        test_loss = criterion(z, y)
        self.log("test_loss", test_loss)
        
        accuracy = torch.sum(one_hot_to_label(z) == one_hot_to_label(y)).item() / len(y)
        self.log("accuracy", accuracy)
        
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        return self(batch)
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer

model = RadarGestureNet( gesture_class=gesture_classes)
summary(model, input_size=[(30, 64),(30, 64),(30, 64)],device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         LayerNorm-1               [-1, 30, 64]           3,840
            Conv1d-2               [-1, 30, 64]             120
            Conv1d-3               [-1, 15, 64]             465
DepthwiseSeparableConv1d-4               [-1, 15, 64]               0
              ReLU-5               [-1, 15, 64]               0
            Conv1d-6               [-1, 10, 64]             160
            Conv1d-7               [-1, 10, 64]             460
            Conv1d-8               [-1, 10, 64]             760
            Conv1d-9               [-1, 10, 64]             160
  InceptionModule-10               [-1, 40, 64]               0
             ReLU-11               [-1, 40, 64]               0
        LayerNorm-12               [-1, 30, 64]           3,840
           Conv1d-13               [-1, 30, 64]             120
           Conv1d-14             

In [147]:
# 训练模型
# -------------------
# 步骤 3: 训练
# -------------------
# autoencoder = LitAutoEncoder()

model = RadarGestureNet(gesture_class=gesture_classes)
summary(model, input_size=[(30, 64),(30, 64),(30, 64)],device="cpu")
trainer = L.Trainer(
    max_epochs=75,
    log_every_n_steps=1,
    deterministic=True,
    enable_progress_bar=False,
    enable_model_summary=False,
    )

train_data_loader = data.DataLoader(train_loader, batch_size=512, shuffle=True)
val_data_loader = data.DataLoader(val_loader, batch_size=512, shuffle=False)

trainer.fit(model, train_data_loader, val_data_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         LayerNorm-1               [-1, 30, 64]           3,840
            Conv1d-2               [-1, 30, 64]             120
            Conv1d-3               [-1, 15, 64]             465
DepthwiseSeparableConv1d-4               [-1, 15, 64]               0
              ReLU-5               [-1, 15, 64]               0
            Conv1d-6               [-1, 10, 64]             160
            Conv1d-7               [-1, 10, 64]             460
            Conv1d-8               [-1, 10, 64]             760
            Conv1d-9               [-1, 10, 64]             160
  InceptionModule-10               [-1, 40, 64]               0
             ReLU-11               [-1, 40, 64]               0
        LayerNorm-12               [-1, 30, 64]           3,840
           Conv1d-13               [-1, 30, 64]             120
           Conv1d-14             

k:\aio_radar\.conda\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
k:\aio_radar\.conda\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
`Trainer.fit` stopped: `max_epochs=75` reached.


In [148]:
# 验证损失
trainer.test(model, data.DataLoader(val_loader,batch_size=256))
trainer.test(model, data.DataLoader(test_loader,batch_size=256))

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


k:\aio_radar\.conda\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.9436666369438171     │
│         test_loss         │   0.017185678705573082    │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.9446666836738586     │
│         test_loss         │   0.016482500359416008    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.016482500359416008, 'accuracy': 0.9446666836738586}]

In [149]:
# 保存模型
# -------------------
# 步骤 4: 保存模型
# -------------------
# onnx
# 保存成pth
torch.save(model.state_dict(), 'model.pth')

model = RadarGestureNet(gesture_class=gesture_classes)
model.load_state_dict(torch.load('model.pth'))
model.eval()
dummy_input = torch.randn(1, 30, 64), torch.randn(1, 30, 64), torch.randn(1, 30, 64)
torch.onnx.export(model, dummy_input, 'avg_model.onnx', verbose=True)


: 

In [50]:
# 加载模型
# model_path = r'K:\aio_radar\lightning_logs\version_85\checkpoints\epoch=74-step=1050.ckpt'
# model = RadarGestureNet.load_from_checkpoint(model_path).to("cpu")
torch.save(model.state_dict(), 'mti_model.pth')
# 加载pth模型
model_path = r'model.pth'
model = RadarGestureNet(gesture_class=gesture_classes)
model.load_state_dict(torch.load(model_path))

# # 预测
predictions = model(torch.rand(range_profile.shape[0],30,64),torch.rand(range_profile.shape[0],30,64),torch.rand(range_profile.shape[0],30,64))
# 
# predictions = model(torch.rand(1,30,64),torch.rand(1,30,64),torch.rand(1,30,64))


predictions = model(range_profile, speed_profile, angle_profile)
print(predictions)

# # 准确率
print(torch.sum(one_hot_to_label(predictions) == one_hot_to_label(labels)).item() / len(labels))


tensor([[9.9118e-01, 5.0276e-05, 1.7751e-07,  ..., 1.0608e-06, 2.2769e-06,
         4.1550e-06],
        [1.0000e+00, 1.2611e-07, 1.7854e-08,  ..., 8.8934e-10, 7.4204e-09,
         5.4652e-08],
        [9.9997e-01, 2.8686e-06, 6.1561e-06,  ..., 3.2462e-07, 8.4617e-07,
         2.5539e-06],
        ...,
        [4.6524e-04, 4.8588e-04, 1.3343e-03,  ..., 2.4570e-06, 9.8908e-06,
         1.4300e-06],
        [9.8021e-03, 2.0627e-03, 3.2580e-07,  ..., 9.9225e-07, 1.0006e-06,
         5.3378e-07],
        [4.9791e-03, 6.4650e-05, 5.5307e-05,  ..., 1.2257e-06, 2.1895e-06,
         5.3736e-07]], grad_fn=<SoftmaxBackward0>)
0.9765


In [8]:
# print(range_profile.shape)
# # 绘制range_profile
# import matplotlib.pyplot as plt
# plt.plot(range_profile[0,3,:])
# for i in range(range_profile.shape[0]):
#     for j in range(range_profile.shape[1]):
#         plt.plot(range_profile[i,j,:])
# plt.show()
    